In [1]:
!pip install transformers datasets scikit-learn torch

In [2]:
import torch
from transformers import BertModel, BertPreTrainedModel
from torch import nn

class BERTCNN(BertPreTrainedModel):
    def __init__(self, config):
        super(BERTCNN, self).__init__(config)
        self.bert = BertModel.from_pretrained("bert-base-uncased", config=config)
        self.conv = nn.Conv1d(in_channels=config.hidden_size, out_channels=128, kernel_size=5, padding=2)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(128, config.num_labels)
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask, inputs_embeds=inputs_embeds)
        sequence_output = outputs[0]
        sequence_output = sequence_output.permute(0, 2, 1)
        x = self.conv(sequence_output)
        x = self.pool(x).squeeze(-1)
        x = self.dropout(x)
        logits = self.classifier(x)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else logits

In [22]:
from transformers import AutoModelForSequenceClassification

class RoBERTa:
    def __init__(self, model_type='cardiffnlp/twitter-roberta-base-sentiment', num_labels=3):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_type, num_labels=num_labels)

    def get_model(self):
        return self.model

In [3]:
!pip install transformers[torch] accelerate -U

In [21]:
import pandas as pd
from transformers import AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, BertConfig
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch

def load_data():
      df = pd.read_csv('Reviews.csv', on_bad_lines='warn', nrows=10000)
      df = df.head(10000)

      def map_score_to_sentiment(score):
          if score < 3:
              return 0  # Negative
          elif score == 3:
              return 1  # Neutral
          else:
              return 2  # Positive

      df['Sentiment'] = df['Score'].apply(map_score_to_sentiment)

      train_texts, test_texts, train_labels, test_labels = train_test_split(
          df['Text'], df['Sentiment'], test_size=0.9, random_state=42)

      train_df = pd.DataFrame({'text': train_texts, 'label': train_labels})
      test_df = pd.DataFrame({'text': test_texts, 'label': test_labels})

      return train_df, test_df


def tokenize_data(model_type, train_df, test_df):
    tokenizer = AutoTokenizer.from_pretrained(model_type)

    def tokenize_function(examples):
        return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

    train_dataset = Dataset.from_pandas(train_df).map(tokenize_function, batched=True)
    test_dataset = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

    return train_dataset, test_dataset, tokenizer

def train_model(model, train_dataset, test_dataset, tokenizer, output_dir):
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        save_total_limit=2,
        save_steps=500,
        eval_strategy="steps",
        eval_steps=500,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=data_collator
    )

    trainer.train()

if __name__ == "__main__":

    train_df, test_df = load_data()

    # BERT-CNN
    bert_model_type = 'bert-base-uncased'
    config = BertConfig.from_pretrained(bert_model_type, num_labels=3)
    train_dataset, test_dataset, tokenizer = tokenize_data(bert_model_type, train_df, test_df)
    bert_cnn_model = BERTCNN(config=config)
    train_model(bert_cnn_model, train_dataset, test_dataset, tokenizer, './bert_cnn_results')

    # Custom RoBERTa
    roberta_model_type = 'cardiffnlp/twitter-roberta-base-sentiment'
    train_dataset, test_dataset, tokenizer = tokenize_data(roberta_model_type, train_df, test_df)
    roberta = RoBERTa(model_type=roberta_model_type)
    roberta_model = roberta.get_model()
    train_model(roberta_model, train_dataset, test_dataset, tokenizer, './roberta_results')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss


KeyboardInterrupt: 